In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import datatable as dt
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices


In [ ]:
TRAINING = True
USE_FINETUNE = False     
FOLDS = 5
SEED = 42
action_type = 1
y_type = 0
ENCODER = False
NANFILL = -9999

train_data_datatable = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv')
train = train_data_datatable.to_pandas()

train = train.query('date > 85').reset_index(drop = True) 
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
fillnaDict = train.mean()

#train.fillna(fillnaDict,inplace=True)
train = train.fillna(NANFILL)

train = train.query('weight > 0').reset_index(drop = True)





if action_type == 0:
    train['action'] = (train['resp'] > 0).astype('int')
elif action_type == 1:
    train['action'] =  (  (train['resp_1'] > 0.00001 ) & (train['resp_2'] > 0.00001 ) & (train['resp_3'] > 0.00001 ) & (train['resp_4'] > 0.00001 ) &  (train['resp'] > 0.00001 )   ).astype('int')
else:
    train['action'] = (train['resp'] > 0).astype('int')
    
datalength = len(train)
trainlength = int(datalength * 0.9)

train, valid = train.loc[:trainlength,:],train.loc[trainlength:,:]
SAMPLE_WEIGHTS = (train['resp'] * train['weight']).abs() + 1

features = [c for c in train.columns if 'feature' in c]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X = train[features].values
Xvalid = valid[features].values

if y_type == 0:
    y      = np.stack([(train[c] > 0.000001).astype('int') for c in resp_cols]).T #Multitarget
    yvalid = np.stack([(valid[c] > 0.000001).astype('int') for c in resp_cols]).T #Multitarget
elif y_type == 1:
    y      = np.stack([ train['action'].values for c in resp_cols]).T #Multitarget
    yvalid = np.stack([ valid['action'].values for c in resp_cols]).T #Multitarget
else:
    y      = np.stack([(train[c] > 0.000001).astype('int') for c in resp_cols]).T #Multitarget
    yvalid = np.stack([(valid[c] > 0.000001).astype('int') for c in resp_cols]).T #Multitarget

f_mean = np.mean(train[features[1:]].values,axis=0)

In [ ]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
import random
from sklearn import metrics

def create_clf():
    clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=500, max_depth=11, max_bin=63,
                         learning_rate=0.05, n_estimators=500,  
                         objective='binary', class_weight='balanced', min_split_gain=0.05, min_child_weight=0.01, 
                         min_child_samples=100, subsample=0.9, subsample_freq=3, colsample_bytree=0.7, 
                         reg_alpha=100, reg_lambda=1.5, random_state=None, n_jobs=-1, 
                         silent=True, importance_type='split',device="cpu")
    '''
    clf =  xgb.XGBClassifier(   n_estimators=50,use_label_encoder=False,eval_metric='logloss',
                                max_depth=6,
                                learning_rate=0.05,
                                subsample=0.9,
                                colsample_bytree=0.7,
                                missing=-999,
                                random_state=random.randint(5000,6000)
                            )
    clf =  LogisticRegression(  C=0.1
                            )
    '''
    return clf

FOLDS = 5
SEED = 42
#gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
#splits = list(gkf.split(y, groups=train['date'].values))



In [ ]:
models = {}
respTrainList = [0,1,2,3,4]
for resp_type in respTrainList:
    clf = create_clf()
    clf.fit(X, y[:,resp_type], sample_weight=SAMPLE_WEIGHTS)
    y_train_prob = clf.predict_proba(X)[:,1]
    auctrain = metrics.roc_auc_score(y[:,resp_type], y_train_prob)

    y_valid_prob = clf.predict_proba(Xvalid)[:,1]
    aucvalid = metrics.roc_auc_score(yvalid[:,resp_type], y_valid_prob)

    print('resp = ',resp_cols[resp_type],auctrain,aucvalid)

    models[f'{resp_cols[resp_type]}'] = clf

In [ ]:
def model_predict_slow(X,models):
    y_multi_pre = concat_respx_pred(X,models)
    y = models['lr'].predict(y_multi_pre)
    #y = y > np.mean(y)
    #print(np.mean(y))
    return y.astype(int)

def model_predict_fast(X,models):
    
    y = models['resp'].predict(X)
    #y = y > np.mean(y)
    #print(np.mean(y))
    return y.astype(int)

In [ ]:
def concat_respx_pred(X,models):
    y_multi_pre = np.stack([ models[f'{respx}'].predict_proba(X)[:,1] for respx in models if respx != 'lr']).T
    return y_multi_pre
lr = LogisticRegression(C=1.0, solver='sag', class_weight={0:0.3,1:0.7})

y_multi_pre = concat_respx_pred(X, models)
yvalid_multi_pre = concat_respx_pred(Xvalid, models)

lr.fit(y_multi_pre, train['action'].values)
models['lr'] = lr

np.mean(y_multi_pre,axis=0),np.mean(lr.predict_proba(y_multi_pre)[:,1]),np.mean(lr.predict(y_multi_pre)),np.mean(train['action'].values)

In [ ]:
train_proba = lr.predict_proba(y_multi_pre)[:,1]
train_pred = model_predict_slow(X,models)
action_auc_train = metrics.roc_auc_score(train['action'], train_proba)
action_mse_train = metrics.accuracy_score(train['action'], train_pred)
print(action_auc_train,action_mse_train)

valid_proba = lr.predict_proba(yvalid_multi_pre)[:,1]
valid_pred = model_predict_slow(Xvalid,models)
action_auc_valid = metrics.roc_auc_score(valid['action'], valid_proba)
action_mse_valid = metrics.accuracy_score(valid['action'], valid_pred)
print(action_auc_valid,action_mse_valid)

In [ ]:
import janestreet
from tqdm import tqdm
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set
#fillnaDict = train.mean()
model_predict = model_predict_slow
features = [f'feature_{x}' for x in range(130)]
for (test_df, sample_prediction_df) in tqdm(iter_test):
    if test_df['weight'].values[0] < 0.001:
        y_preds = 0
    else:
        X_test = test_df.loc[:, features].fillna(0.0).values
        y_preds = model_predict(X_test,models)
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)

In [ ]:
df = pd.read_csv('submission.csv')
df.describe()

In [ ]:
!ls /kaggle/input/models